<a href="https://colab.research.google.com/github/dmitry-kabanov/datascience/blob/main/2022-06-24-lstm-in-keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using LSTM in Keras and Tensorflow 2

Based on the materials by Christian Versloot from here: https://github.com/christianversloot/machine-learning-articles/blob/main/build-an-lstm-model-with-tensorflow-and-keras.md#full-model-code

In [7]:
import tensorflow as tf

from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Model configuration
additional_metrics = ["accuracy"]
batch_size = 128
embedding_output_dims = 15
loss_fn = BinaryCrossentropy()
max_sequence_length = 300
num_distinct_words = 5000
epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1


## Load and prepare data

In [9]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
print(f"x_train.shape: {x_train.shape}")
print(f"x_test.shape: {x_test.shape}")

17473536/17464789 [==============================] - 0s 0us/step
x_train.shape: (25000,)
x_test.shape: (25000,)


In [10]:
# Pad all sequences.
pad_train = pad_sequences(x_train, maxlen=max_sequence_length, value=0.0)
pad_test = pad_sequences(x_test, maxlen=max_sequence_length, value=0.0)

## Define the model

In [11]:
model = Sequential([
    Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length),
    LSTM(10),
    Dense(1, activation="sigmoid"),
])

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=additional_metrics
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 15)           75000     
                                                                 
 lstm (LSTM)                 (None, 10)                1040      
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 76,051
Trainable params: 76,051
Non-trainable params: 0
_________________________________________________________________


## Training

In [13]:
hist = model.fit(
    pad_train,
    y_test,
    batch_size=batch_size,
    epochs=epochs,
    verbose=verbosity_mode,
    validation_split=validation_split
)


Epoch 1/5
157/157 [==============================] - 23s 146ms/step - loss: 0.6900 - accuracy: 0.5623 - val_loss: 0.6876 - val_accuracy: 0.5620
Epoch 2/5
157/157 [==============================] - 24s 153ms/step - loss: 0.6762 - accuracy: 0.5791 - val_loss: 0.6910 - val_accuracy: 0.5384
Epoch 3/5
157/157 [==============================] - 22s 143ms/step - loss: 0.6249 - accuracy: 0.6522 - val_loss: 0.7281 - val_accuracy: 0.5280
Epoch 4/5
157/157 [==============================] - 22s 140ms/step - loss: 0.5626 - accuracy: 0.7115 - val_loss: 0.7896 - val_accuracy: 0.5158
Epoch 5/5
157/157 [==============================] - 22s 142ms/step - loss: 0.5102 - accuracy: 0.7506 - val_loss: 0.8348 - val_accuracy: 0.5194


## Test performance

In [14]:
test_results = model.evaluate(pad_test, y_test, verbose=False)
print(f"Test results")
print(f"Loss: {test_results[0]}, Accuracy: {100*test_results[1]}")

Test results
Loss: 0.8681398630142212, Accuracy: 49.36400055885315
